# Power SE Michigan Project Map

This notebook has been designed to render Power project data into a geomap. Active projects will be rendered as markers on an interactive map with a focus on the SE Michigan area. The map will be saved to an HTML file to be opened in browser of choice. 

So long as column names and data formats stay conistent, this notebook can be run on as needed with new input files. 

### Install dependencies


### Import Libraries


In [5]:
from distutils.log import error
from multiprocessing.context import _default_context
import os
import pandas as pd
from geopy.geocoders import Nominatim
import folium
from branca.element import Figure

### Read in data
If using a new file, update the file name in the cell below. 

In [6]:
file_dir = os.path.join(os.getcwd(), "./files/Untitled 18.csv")

df = pd.read_csv(file_dir)

### Clean Data
Remove whitespace, newlines, and `DET` from end of addresses

In [7]:
df = df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True).replace(r'\bDET\b', '', regex=True)

### Create map
Generate Folium map with coordinates of addresses pulled down from imput file. Markers will be labeled by project number and display additional data such as `Date`, `Owner`, `Home`, `Product`, `Rep`, and `Price` when selected. 

In [8]:
fig = Figure(width=550, height=400)
map = folium.Map(location=[42.3314, -83.0458], zoom_start=10)
fig.add_child(map)

folium.TileLayer('Stamen Terrain').add_to(map)
folium.TileLayer('Stamen Toner').add_to(map)
folium.TileLayer('Stamen Water Color').add_to(map)
folium.TileLayer('cartodbpositron').add_to(map)
folium.TileLayer('cartodbdark_matter').add_to(map)
folium.LayerControl().add_to(map)

geolocater = Nominatim(user_agent="tutorial")
for index, row in df.iterrows():
    try:
        location = geolocater.geocode(row["Home"])
        if location.latitude is not None:
            folium.Marker(
                location=[location.latitude, location.longitude],
                popup=f"""
                <table>
                    <tr>
                        <th>Date</th>
                        <th>Owner</th>
                        <th>Home</th>
                        <th>Product</th>
                        <th>Rep</th>
                        <th>Price</th>
                    </tr>
                    <tr>
                        <td>{row["Date"]}</td>
                        <td>{row["Owner"]}</td>
                        <td>{row["Home"]}</td>
                        <td> {row["Product"]}</td>
                        <td>{row["Rep"]}</td>
                        <td>{row["Price"]}</td>
                    </tr>
                </table>
                """,
                tooltip=f'Project # {row["Project #"]}'
            ).add_to(map)
    except:
        pass
        
map

### Save the map

The map will be saved as an html file which can then be opened in a browser. The filename can be updated in this cell for any future versions of the map you wish to create

In [9]:
map.save('power-SE-MI.html')